In [1]:
from dash import dcc, html, Output, Input
from jupyter_dash import JupyterDash
import plotly.express as px
import base64, pandas as pd

In [64]:
# specify the directory of the project
proj_dir = '../'

# read the data that matches the specified DateTime to the location of the image
df = pd.read_csv(f'{proj_dir}data/bkk_all.csv').drop('Unnamed: 0', axis=1)
df.head()

,DateTime,FolderPath,FileName
0,2020-06-18 06:30:00,/data/bkk_radar_images/bkk_radar_images_1/,1592461954.png
1,2020-06-18 06:35:00,/data/bkk_radar_images/bkk_radar_images_1/,1592462136.png
2,2020-06-18 06:45:00,/data/bkk_radar_images/bkk_radar_images_1/,1592462768.png
3,2020-06-18 06:50:00,/data/bkk_radar_images/bkk_radar_images_1/,1592463068.png
4,2020-06-18 06:55:00,/data/bkk_radar_images/bkk_radar_images_1/,1592463277.png


In [65]:
# extract the DateTime column
DateTime = df['DateTime']
DateTime.head(3)

0    2020-06-18 06:30:00
1    2020-06-18 06:35:00
2    2020-06-18 06:45:00
Name: DateTime, dtype: object

In [66]:
# extract the Date
Date = DateTime.str.split(' ').str[0].unique()

In [67]:
# extract the timestamp and create a dictionary
Time = {}
for date in Date:
    Time[date] = DateTime[DateTime.str.contains(date)].str.split(' ').str[1]

In [106]:
# Dash section
app = JupyterDash(__name__)

app.layout = html.Div([

    html.Div([
        html.Div([
            html.H1('Nowcasting', style={'color': '#FFFFFF'}),

            html.Div([
                html.Div([

                    html.P('Date: ', style={'color': '#FFFFFF'}),
                    dcc.Dropdown(
                        id='date-dropdown',
                        options=[{'label': i, 'value': i} for i in Date],
                        value=Date[0],
                        style={'width': '150px'}
                    ),
                ]),
                html.Div([    
                    html.P('Time: ', style={'color': '#FFFFFF'}),
                    dcc.Dropdown(id='time-dropdown', style={'width': '150px'})
                ])
            ], style={'display': 'flex', 'gap': '10px'}),

            ], 
        style={'display': 'flex', 'flexDirection': 'column'}),

        html.Div([
            html.Img(id='image', style={'width': '100%', 
                                    'height': '100%', 
                                    'backgroundColor': '#FFFFFF'}),], 
                        style={'display': 'flex', 'justify-content': 'center'}),
        html.Div([
            html.Img(id='image-2', style={'width': '100%', 
                                    'height': '100%', 
                                    'backgroundColor': '#FFFFFF'}),], 
                        style={'display': 'flex', 'justify-content': 'center'}),
        html.Div([
            html.Img(id='image-3', style={'width': '100%', 
                                    'height': '100%', 
                                    'backgroundColor': '#FFFFFF'}),], 
                        style={'display': 'flex', 'justify-content': 'center'}),

    ], style={'display': 'grid', 
                'grid-template-columns': 'repeat(2, 2fr)',
                'grid-template-rows': ['10px', '10px'],
                'grid-gap': '10px',
                'justify-content': 'center',
                'align-items': 'center',
                'margin-top': '10px'}),
])

@app.callback(
    Output('time-dropdown', 'options'),
    Output('time-dropdown', 'value'),
    Input('date-dropdown', 'value')
)
def updateTimeDropdown(date):
    return [{'label': i, 'value': i} for i in Time[date]], Time[date].iloc[0]

@app.callback(
    Output('image', 'src'),
    Output('image-2', 'src'),
    Output('image-3', 'src'),
    Input('date-dropdown', 'value'),
    Input('time-dropdown', 'value'),
)
def updateImg(date, time):
    result = df.loc[df['DateTime'] == f'{date} {time}', ['FolderPath', 'FileName']]
    path = '../' + result.iloc[0,0] + result.iloc[0,1]
    encoded_image = base64.b64encode(open(path, 'rb').read())
    ret = 'data:image/png;base64,{}'.format(encoded_image.decode())
    return (ret, ret, ret)

if __name__ == '__main__':
    app.run_server(mode='inline')